In [7]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, DuckDuckGoSearchResults
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
  query: str = Field(description="The query you will search for. Example query: Stock Market Symbol for Apple Company")

class StockMarketSymbolSearchTool(BaseTool):
  name = "StockMarketSymbolSearchTool"
  description= """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    """
  args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema
  
  def _run(self, query):
    ddg = DuckDuckGoSearchResults()
    return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
  symbol: str = Field(description="Stock symbol of the company. Example: AAPL, TSLA")

class CompanyOverviewTool(BaseTool):
  name = "CompanyOverview"
  description = """
  Use this ti get an overview of the financials of the company.
  You should enter a stock symbol.
  """
  args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

  def _run(self, symbol):
    r = requests.get(f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}")
    return r.json()


class CompanyIncomeStatementTool(BaseTool):
  name = "CompanyIncomeStatement"
  description = """
  Use this ti get an income statement of a company.
  You should enter a stock symbol.
  """
  args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

  def _run(self, symbol):
    r = requests.get(f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")
    return r.json()["annualReports"]
  

class CompanyStockPerformanceTool(BaseTool):
  name = "CompanyStockPerformance"
  description = """
  Use this ti get the daily performance of a company stock.
  You should enter a stock symbol.
  """
  args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

  def _run(self, symbol):
    r = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}")
    return r.json()


agent = initialize_agent(
  llm=llm, 
  verbose=True,
  agent=AgentType.OPENAI_FUNCTIONS,
  handle_parsing_errors=True,
  tools=[
    StockMarketSymbolSearchTool(),
    CompanyOverviewTool(),
    CompanyIncomeStatementTool(),
    CompanyStockPerformanceTool()
  ]
)

prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Cloudflare'}`


[snippet: Stock analysis for Cloudflare Inc (NET:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile., title: NET: Cloudflare Inc Stock Price Quote - New York - Bloomberg, link: https://www.bloomberg.com/quote/NET:US], [snippet: About NET. Cloudflare, Inc. operates as a cloud services provider that delivers a range of services to businesses worldwide. The company provides an integrated cloud-based security solution to secure a range of combination of platforms, including public cloud, private cloud, on-premise, software-as-a-service applications, and IoT devices; and website and application security products ..., title: Cloudflare, Inc. (NET) Stock Price, Quote & News - Stock Analysis, link: https://stockanalysis.com/stocks/net/], [snippet: Real-time Cloudflare stock price (NYSE: NET) ... Cloud

{'input': "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment.",
 'output': "### Cloudflare Inc (NET) Financial Information:\n\n#### Company Overview:\n- **Description:** CloudFlare, Inc. operates a cloud platform that offers a range of network services to companies worldwide.\n- **Sector:** Technology\n- **Industry:** Services-Prepackaged Software\n- **Market Capitalization:** $32.75 billion\n- **Revenue (TTM):** $1.3 billion\n- **EPS (TTM):** -$0.55\n- **Profit Margin:** -14.2%\n- **Operating Margin (TTM):** -11.8%\n- **Return on Assets (TTM):** -4.34%\n- **Return on Equity (TTM):** -26.5%\n- **Analyst Target Price:** $100.9\n- **52-Week High:** $116\n- **52-Week Low:** $39.9\n\n#### Income Statement (2023):\n- **Total Revenue:** $1.3 billion\n- **Gross Profit:** $989.74 million\n- **Operating Income:** -$185.48 million\n- **Net Income:** -$183.95 million\n\n#### 